# I. Lý thuyết

1) Tại sao các mô hình deep learning lại chiếm ưu thế hơn so với các mô hình machine learning truyền thống đối với dữ liệu lớn ?

A. Do mô hình deep learning có thể được thiết kế với kích thước tùy ý nên có khả năng xấp xỉ mọi hàm số. Do đó nó có khả năng biểu diễn tốt và hoạt động hiệu quả trên dữ liệu lớn.

B. Các mô hình machine learning thường bị overfitting đối với dữ liệu lớn ?

C. Các mô hình deep learning có chi phí huấn luyện tốn kém hơn so với machine learning.

D. Do kiến trúc của mô hình Machine Learning bao gồm nhiều layers xếp chồng.


2) Ý nghĩa của hàm loss function trong mạng neural network là gì ?

A. Là hàm số đánh giá độ chính xác của mô hình.

B. Mục tiêu của quá trình huấn luyện là tối thiểu hóa hàm loss function bằng thuật toán gradient descent. Giá trị của hàm số này giúp đo lường mức độ khớp của dự báo từ mô hình trên dữ liệu huấn luyện.

C. Khi loss function giảm thì luôn đảm bảo độ chính xác của mô hình tăng.

D. Là hàm số cần tối đa hóa trong quá trình huấn luyện.


3) Khi huấn luyện trên các bộ dữ liệu bigdata thì chúng ta nên sử dụng phương pháp nào ?

A) Sử dụng gradient descent trên toàn bộ dữ liệu.

B) Sử dụng stochastic gradient descent trên từng điểm dữ liệu.

C) Mini-batch gradient descent huấn luyện mô hình trên từng tập dữ liệu con có kích thước nhỏ hơn memory CPU/GPU.

D) Có thể sử dụng stochastic gradient descent hoặc mini-batch gradient descent.


4) Quá trình feed forward và backpropagation thực hiện những gì ?

A) feed forward tính toán output và loss function, backpropagation tính đạo hàm trên từng layer và cập nhật trọng số.

B) feed forward cập nhật trọng số cho mô hình, backpropagation tính toán output và loss function.

C) feed forward tính ra output của mô hình, backpropagation tính toán loss function

D) feed forward được thực hiện sau backpropagation.

5) Tác dụng của batch normalization là gì ?

A) Loại bỏ một tỷ lệ ngẫu nhiên số lượng units tại mỗi layer để tạo thành nhiều kiến trúc kết hợp ngẫu nhiên.

B) Tìm ra các tham số phân phối là trung bình và phương sai trên từng mini-batch.

C) Đồng nhất phân phối xác suất của $z^{[l]}$ trên mỗi layer $l$.

D) Giảm thiểu ảnh hưởng của input distribution shift nhằm giúp huấn luyện loss function nhanh và ổn định hơn.

### Câu 1: A

### Câu 2: B

### Câu 3: C

### Câu 4: A

### Câu 5: D

# II. Thực hành

Xuất phát từ mô hình tốt nhất của bạn xây dựng được đối với bài toán phân loại income classification tại bài trước. Bạn hãy thực hiện một số thử nghiệm sau:

6) Thay đổi hàm loss function, batch size và optimizer.

7) Thử nghiệm thêm các layers mà bạn đã học được trong bài này vào kiến trúc của mình.

8) Thay đổi các khởi tạo trọng số theo các phân phối khác nhau và đánh giá độ chính xác của kết quả huấn luyện.

9) Thiết lập không gian search và tự động hóa tìm kiếm kiến trúc tốt nhất trên optuna.

10) Deploy model sử dụng flask ap. Tham khảo [Flaskapp tutorial](https://drive.google.com/file/d/1AZNtzrmnhJ-OBgijWoaAqXbPhJ6xL0Po/view?usp=sharing).

In [ ]:
pip install category_encoders

In [ ]:
pip install optuna

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('gdrive/MyDrive/DeepLearning Course 3')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import torch.utils.data as td
import torch
import torch.nn as nn
import category_encoders as ce
import torch.optim as optim
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

torch.manual_seed(42)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('../train.csv')

In [ ]:
# Xử lý outlier

def find_boxplot_boundaries(feature, whisker_coeff=1.5):
    Q1 = feature.quantile(0.25)
    Q3 = feature.quantile(0.75)
    
    IQR = Q3 - Q1
    lower = Q1 - IQR * whisker_coeff
    upper = Q3 + IQR * whisker_coeff
    
    return lower, upper


class BoxplotOutlierClipper(BaseEstimator, TransformerMixin):
    
    def __init__(self, whisker_coeff=1.5):
        self.whisker_coeff = whisker_coeff
        self.lower = None
        self.upper = None
    
    def fit(self, feature: pd.Series):
        self.lower, self.upper = find_boxplot_boundaries(feature)
        return self
    
    def transform(self, feature):
        return feature.clip(self.lower, self.upper)
    
    
df['final_weight'] = BoxplotOutlierClipper().fit_transform(df['final_weight'])
df['hrs_per_week'] = BoxplotOutlierClipper().fit_transform(df['hrs_per_week'])
df['total_education_yrs'] = BoxplotOutlierClipper().fit_transform(df['total_education_yrs'])

In [ ]:
# Những quốc gia có số lượng ít hơn 100 chuyển thành other
def convert_nationality(x):
    if x=='US':
        return x
    elif x=='Mexico':
        return x
    elif x=='?':
        return x
    elif x=='Philippines':
        return x
    elif x=='Germany':
        return x
    else:
        return 'other'

In [ ]:
# xóa bỏ những giá trị có captital gain từ 40000 tới 100000 (chính là giá trị 99999)
df = df[~(df['capital_gain'] >= 40000) & (df['capital_gain'] <= 100000)]

In [ ]:
def fe(X_train, X_val):
    cols = df.select_dtypes(include='object')
    onehot = ce.OneHotEncoder(cols)
    X_train = onehot.fit_transform(X_train)
    X_val = onehot.transform(X_val)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    
    return X_train, X_val    

In [ ]:
df_new = df.sample(frac=1, ignore_index=True, random_state=42)

X = df_new.drop(['ID', 'target_income'], axis=1)
y = df_new['target_income']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val = fe(X_train, X_val)

## Câu 6

In [ ]:
# thay đổi batch_size=16

X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train.values).long()
train_ds = td.TensorDataset(X_train, y_train)
train_loader = td.DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=1)

X_val = torch.tensor(X_val).float()
y_val = torch.tensor(y_val.values).long()
val_ds = td.TensorDataset(X_val, y_val)
val_loader = td.DataLoader(val_ds, batch_size=16, num_workers=1)



In [ ]:
class MyModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(X_train.size()[1], 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 10)
        self.fc4 = nn.Linear(10, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

model = MyModel()
model

MyModel(
  (fc1): Linear(in_features=108, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=2, bias=True)
)

In [ ]:
# thay đổi loss function và optimizer

DEVICE = torch.device("cpu")
LEARNING_RATE = 0.01
EPOCHS = 100
LOSS_CRITERIA = nn.CrossEntropyLoss()
MAX_ACC = -float('inf')
ERROR_GOING_UP = 0
OPTIMIZER = torch.optim.RMSprop(model.parameters(), lr=LEARNING_RATE)

def train(model, data_loader, optimizer, loss_criteria):
    model.train()
    losses = 0
    for batch, (data, target) in enumerate(data_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        out = model(data)
        loss = loss_criteria(out, target)
        losses += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_loss = losses / (batch + 1)
    print(f"Loss in train set: {avg_loss:.6f}")
    return avg_loss

def eval(model, data_loader, optimizer, loss_criteria):
    model.eval()
    losses = 0
    acc = 0
    with torch.no_grad():
        for batch, (data, target) in enumerate(data_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            out = model(data)
            loss = loss_criteria(out, target)
            losses += loss.item()
            _, predict = torch.max(out.data, axis=-1)
            acc += predict.eq(target).sum().item()
    avg_loss = losses / (batch + 1)
    acc = acc * 100 / len(data_loader.dataset)
    print(f"Loss in validation set: {avg_loss:.6f}, Accuracy {acc:.3f}%")
    print('-'*50)
    return avg_loss, acc

for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}')
    train_loss = train(model, train_loader, OPTIMIZER, LOSS_CRITERIA)
    val_loss, acc = eval(model, val_loader, OPTIMIZER, LOSS_CRITERIA)

    if acc > MAX_ACC:
        MAX_ACC = acc
        ERROR_GOING_UP = 0
    else:
        ERROR_GOING_UP += 1
        if ERROR_GOING_UP == 7:
            break
print(f"Accuracy: {MAX_ACC:.3f}")

Epoch 1
Loss in train set: 0.465883
Loss in validation set: 0.454548, Accuracy 85.212%
--------------------------------------------------
Epoch 2
Loss in train set: 0.455059
Loss in validation set: 0.458043, Accuracy 83.605%
--------------------------------------------------
Epoch 3
Loss in train set: 0.452820
Loss in validation set: 0.454828, Accuracy 85.031%
--------------------------------------------------
Epoch 4
Loss in train set: 0.453779
Loss in validation set: 0.450096, Accuracy 85.533%
--------------------------------------------------
Epoch 5
Loss in train set: 0.449484
Loss in validation set: 0.451400, Accuracy 84.890%
--------------------------------------------------
Epoch 6
Loss in train set: 0.448029
Loss in validation set: 0.451688, Accuracy 85.252%
--------------------------------------------------
Epoch 7
Loss in train set: 0.448486
Loss in validation set: 0.453600, Accuracy 84.931%
--------------------------------------------------
Epoch 8
Loss in train set: 0.50043

## Câu 7

In [ ]:
class MyModelQ7(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(X_train.size()[1], 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 10)
        self.fc4 = nn.Linear(10, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

model_q7 = MyModelQ7()

DEVICE = torch.device("cpu")
LEARNING_RATE = 0.01
EPOCHS = 100
LOSS_CRITERIA = nn.CrossEntropyLoss()
MAX_ACC = -float('inf')
ERROR_GOING_UP = 0
OPTIMIZER = torch.optim.RMSprop(model_q7.parameters(), lr=LEARNING_RATE)

def train(model, data_loader, optimizer, loss_criteria):
    model.train()
    losses = 0
    for batch, (data, target) in enumerate(data_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        out = model(data)
        loss = loss_criteria(out, target)
        losses += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_loss = losses / (batch + 1)
    print(f"Loss in train set: {avg_loss:.6f}")
    return avg_loss

def evaluation(model, data_loader, optimizer, loss_criteria):
    model.eval()
    losses = 0
    acc = 0
    with torch.no_grad():
        for batch, (data, target) in enumerate(data_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            out = model(data)
            loss = loss_criteria(out, target)
            losses += loss.item()
            _, predict = torch.max(out.data, axis=-1)
            acc += predict.eq(target).sum().item()
    avg_loss = losses / (batch + 1)
    acc = acc * 100 / len(data_loader.dataset)
    print(f"Loss in validation set: {avg_loss:.6f}, Accuracy {acc:.3f}%")
    print('-'*50)
    return avg_loss, acc

for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}')
    train_loss = train(model_q7, train_loader, OPTIMIZER, LOSS_CRITERIA)
    val_loss, acc = evaluation(model_q7, val_loader, OPTIMIZER, LOSS_CRITERIA)

    if acc > MAX_ACC:
        MAX_ACC = acc
        ERROR_GOING_UP = 0
    else:
        ERROR_GOING_UP += 1
        if ERROR_GOING_UP == 7:
            break
print(f"Accuracy: {MAX_ACC:.3f}")

Epoch 1
Loss in train set: 0.472211
Loss in validation set: 0.463843, Accuracy 84.288%
--------------------------------------------------
Epoch 2
Loss in train set: 0.460987
Loss in validation set: 0.460270, Accuracy 84.810%
--------------------------------------------------
Epoch 3
Loss in train set: 0.459868
Loss in validation set: 0.459252, Accuracy 84.368%
--------------------------------------------------
Epoch 4
Loss in train set: 0.461653
Loss in validation set: 0.463527, Accuracy 84.690%
--------------------------------------------------
Epoch 5
Loss in train set: 0.462470
Loss in validation set: 0.463754, Accuracy 85.152%
--------------------------------------------------
Epoch 6
Loss in train set: 0.462590
Loss in validation set: 0.459368, Accuracy 84.931%
--------------------------------------------------
Epoch 7
Loss in train set: 0.460787
Loss in validation set: 0.460105, Accuracy 84.669%
--------------------------------------------------
Epoch 8
Loss in train set: 0.45895

## Câu 8

In [ ]:
# weights with kaiming_uniform_

class MyModelQ8(nn.Module):
    
    def __init__(self, init_weight=None):
        super().__init__()
        self.fc1 = nn.Linear(X_train.size()[1], 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 10)
        self.fc4 = nn.Linear(10, 2)
        
        if init_weight is not None:
            for fc in [self.fc1, self.fc2, self.fc3, self.fc4]:
                if init_weight == 'kaiming_uniform_':
                    nn.init.kaiming_uniform_(fc.weight, nonlinearity='relu')
                elif init_weight == 'kaiming_normal_':
                    nn.init.kaiming_normal_(fc.weight, nonlinearity='relu')

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

model_q8 = MyModelQ8(init_weight='kaiming_uniform_')

DEVICE = torch.device("cpu")
LEARNING_RATE = 0.01
EPOCHS = 100
LOSS_CRITERIA = nn.CrossEntropyLoss()
MAX_ACC = -float('inf')
ERROR_GOING_UP = 0
OPTIMIZER = torch.optim.RMSprop(model_q8.parameters(), lr=LEARNING_RATE)

def train(model, data_loader, optimizer, loss_criteria):
    model.train()
    losses = 0
    for batch, (data, target) in enumerate(data_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        out = model(data)
        loss = loss_criteria(out, target)
        losses += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_loss = losses / (batch + 1)
    print(f"Loss in train set: {avg_loss:.6f}")
    return avg_loss

def evaluation(model, data_loader, optimizer, loss_criteria):
    model.eval()
    losses = 0
    acc = 0
    with torch.no_grad():
        for batch, (data, target) in enumerate(data_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            out = model(data)
            loss = loss_criteria(out, target)
            losses += loss.item()
            _, predict = torch.max(out.data, axis=-1)
            acc += predict.eq(target).sum().item()
    avg_loss = losses / (batch + 1)
    acc = acc * 100 / len(data_loader.dataset)
    print(f"Loss in validation set: {avg_loss:.6f}, Accuracy {acc:.3f}%")
    print('-'*50)
    return avg_loss, acc

for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}')
    train_loss = train(model_q8, train_loader, OPTIMIZER, LOSS_CRITERIA)
    val_loss, acc = evaluation(model_q8, val_loader, OPTIMIZER, LOSS_CRITERIA)

    if acc > MAX_ACC:
        MAX_ACC = acc
        ERROR_GOING_UP = 0
    else:
        ERROR_GOING_UP += 1
        if ERROR_GOING_UP == 7:
            break
print(f"Accuracy: {MAX_ACC:.3f}")

Epoch 1
Loss in train set: 0.467085
Loss in validation set: 0.453303, Accuracy 85.071%
--------------------------------------------------
Epoch 2
Loss in train set: 0.457219
Loss in validation set: 0.454800, Accuracy 85.071%
--------------------------------------------------
Epoch 3
Loss in train set: 0.457227
Loss in validation set: 0.456179, Accuracy 84.850%
--------------------------------------------------
Epoch 4
Loss in train set: 0.456489
Loss in validation set: 0.457961, Accuracy 84.448%
--------------------------------------------------
Epoch 5
Loss in train set: 0.453232
Loss in validation set: 0.454793, Accuracy 84.649%
--------------------------------------------------
Epoch 6
Loss in train set: 0.506342
Loss in validation set: 0.460926, Accuracy 85.051%
--------------------------------------------------
Epoch 7
Loss in train set: 0.453881
Loss in validation set: 0.460384, Accuracy 84.890%
--------------------------------------------------
Epoch 8
Loss in train set: 0.45124

In [ ]:
# weights with kaiming_normal_

class MyModelQ8(nn.Module):
    
    def __init__(self, init_weight=None):
        super().__init__()
        self.fc1 = nn.Linear(X_train.size()[1], 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 10)
        self.fc4 = nn.Linear(10, 2)
        
        if init_weight is not None:
            for fc in [self.fc1, self.fc2, self.fc3, self.fc4]:
                if init_weight == 'kaiming_uniform_':
                    nn.init.kaiming_uniform_(fc.weight, nonlinearity='relu')
                elif init_weight == 'kaiming_normal_':
                    nn.init.kaiming_normal_(fc.weight, nonlinearity='relu')

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

model_q8 = MyModelQ8(init_weight='kaiming_normal_')

DEVICE = torch.device("cpu")
LEARNING_RATE = 0.01
EPOCHS = 100
LOSS_CRITERIA = nn.CrossEntropyLoss()
MAX_ACC = -float('inf')
ERROR_GOING_UP = 0
OPTIMIZER = torch.optim.RMSprop(model_q8.parameters(), lr=LEARNING_RATE)

def train(model, data_loader, optimizer, loss_criteria):
    model.train()
    losses = 0
    for batch, (data, target) in enumerate(data_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        out = model(data)
        loss = loss_criteria(out, target)
        losses += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_loss = losses / (batch + 1)
    print(f"Loss in train set: {avg_loss:.6f}")
    return avg_loss

def evaluation(model, data_loader, optimizer, loss_criteria):
    model.eval()
    losses = 0
    acc = 0
    with torch.no_grad():
        for batch, (data, target) in enumerate(data_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            out = model(data)
            loss = loss_criteria(out, target)
            losses += loss.item()
            _, predict = torch.max(out.data, axis=-1)
            acc += predict.eq(target).sum().item()
    avg_loss = losses / (batch + 1)
    acc = acc * 100 / len(data_loader.dataset)
    print(f"Loss in validation set: {avg_loss:.6f}, Accuracy {acc:.3f}%")
    print('-'*50)
    return avg_loss, acc

for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}')
    train_loss = train(model_q8, train_loader, OPTIMIZER, LOSS_CRITERIA)
    val_loss, acc = evaluation(model_q8, val_loader, OPTIMIZER, LOSS_CRITERIA)

    if acc > MAX_ACC:
        MAX_ACC = acc
        ERROR_GOING_UP = 0
    else:
        ERROR_GOING_UP += 1
        if ERROR_GOING_UP == 7:
            break
print(f"Accuracy: {MAX_ACC:.3f}")

Epoch 1
Loss in train set: 0.498342
Loss in validation set: 0.495917, Accuracy 78.119%
--------------------------------------------------
Epoch 2
Loss in train set: 0.526129
Loss in validation set: 0.548438, Accuracy 76.411%
--------------------------------------------------
Epoch 3
Loss in train set: 0.489991
Loss in validation set: 0.504788, Accuracy 76.411%
--------------------------------------------------
Epoch 4
Loss in train set: 0.497504
Loss in validation set: 0.496085, Accuracy 76.411%
--------------------------------------------------
Epoch 5
Loss in train set: 0.494893
Loss in validation set: 0.485980, Accuracy 76.411%
--------------------------------------------------
Epoch 6
Loss in train set: 0.490719
Loss in validation set: 0.495291, Accuracy 76.411%
--------------------------------------------------
Epoch 7
Loss in train set: 0.490551
Loss in validation set: 0.490971, Accuracy 76.411%
--------------------------------------------------
Epoch 8
Loss in train set: 0.49083

## Câu 9

### Optuna in Pytorch

In [ ]:
import optuna
from optuna.trial import TrialState

In [ ]:
def define_model(trial):
    n_layers = trial.suggest_int('n_layers', 2, 5)
    layers = []
    in_features = X_train.size()[1]
    for i in range(n_layers):
        out_features = trial.suggest_int('n_units_l{}'.format(i), 4, 64)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        prob = trial.suggest_float('dropout_l{}'.format(i), 0.2, 0.5)
        layers.append(nn.Dropout(prob))
        in_features = out_features
    layers.append(nn.Linear(in_features, 2))
    layers.append(nn.Sigmoid())
    
    return nn.Sequential(*layers)

In [ ]:
DEVICE = torch.device("cpu")
BATCHSIZE = 16
EPOCHS = 30

def objective(trial):
    model = define_model(trial).to(DEVICE)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    loss_criteria = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = loss_criteria(output, target)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(val_loader):
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)
                _, pred = torch.max(output, axis=-1)
                correct += pred.eq(target).sum().item()

        accuracy = correct / len(val_loader.dataset)

        trial.report(accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=900)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2022-02-18 10:48:47,051] A new study created in memory with name: no-name-aa9e100b-5ada-49af-83d9-2e56b81e2ffa
[I 2022-02-18 10:51:49,316] Trial 0 finished with value: 0.8555354631303999 and parameters: {'n_layers': 3, 'n_units_l0': 18, 'dropout_l0': 0.3192228380390431, 'n_units_l1': 34, 'dropout_l1': 0.41543260127512643, 'n_units_l2': 33, 'dropout_l2': 0.2176656414722933, 'optimizer': 'Adam', 'lr': 0.0007387434703682037}. Best is trial 0 with value: 0.8555354631303999.
[I 2022-02-18 10:54:50,361] Trial 1 finished with value: 0.7641149286718907 and parameters: {'n_layers': 3, 'n_units_l0': 48, 'dropout_l0': 0.30857972002455647, 'n_units_l1': 8, 'dropout_l1': 0.22862513268774018, 'n_units_l2': 8, 'dropout_l2': 0.3603128784008549, 'optimizer': 'Adam', 'lr': 0.07123216165129294}. Best is trial 0 with value: 0.8555354631303999.
[I 2022-02-18 10:57:21,549] Trial 2 finished with value: 0.8527225236085996 and parameters: {'n_layers': 2, 'n_units_l0': 59, 'dropout_l0': 0.21644544473438812, 

Study statistics: 
  Number of finished trials:  12
  Number of pruned trials:  6
  Number of complete trials:  6
Best trial:
  Value:  0.8573437813944143
  Params: 
    n_layers: 2
    n_units_l0: 35
    dropout_l0: 0.32784109110544923
    n_units_l1: 37
    dropout_l1: 0.282523114441712
    optimizer: RMSprop
    lr: 0.0001417041353571361


In [ ]:
model = define_model(trial)

In [ ]:
def train(model, data_loader, optimizer, loss_criteria):
    model.train()
    losses = 0
    for batch, (data, target) in enumerate(data_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        out = model(data)
        loss = loss_criteria(out, target)
        losses += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_loss = losses / (batch + 1)
    print(f"Loss in train set: {avg_loss:.6f}")
    return avg_loss

def evaluation(model, data_loader, optimizer, loss_criteria):
    model.eval()
    losses = 0
    acc = 0
    with torch.no_grad():
        for batch, (data, target) in enumerate(data_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            out = model(data)
            loss = loss_criteria(out, target)
            losses += loss.item()
            _, predict = torch.max(out.data, axis=-1)
            acc += predict.eq(target).sum().item()
    avg_loss = losses / (batch + 1)
    acc = acc * 100 / len(data_loader.dataset)
    print(f"Loss in validation set: {avg_loss:.6f}, Accuracy {acc:.3f}%")
    print('-'*50)
    return avg_loss, acc

OPTIMIZER = getattr(optim, trial.params['optimizer'])(model.parameters(), lr=trial.params['lr'])
LOSS_CRITERIA = nn.CrossEntropyLoss()
MAX_ACC = -float('inf')
ERROR_GOING_UP = 0
EPOCHS = 30

for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}')
    train_loss = train(model, train_loader, OPTIMIZER, LOSS_CRITERIA)
    val_loss, acc = evaluation(model, val_loader, OPTIMIZER, LOSS_CRITERIA)

    if acc > MAX_ACC:
        MAX_ACC = acc
        ERROR_GOING_UP = 0
    else:
        ERROR_GOING_UP += 1
        if ERROR_GOING_UP == 7:
            break
print(f"Best accuracy: {MAX_ACC:.3f}")

Epoch 1
Loss in train set: 0.542352
Loss in validation set: 0.491281, Accuracy 78.340%
--------------------------------------------------
Epoch 2
Loss in train set: 0.479595
Loss in validation set: 0.464957, Accuracy 84.549%
--------------------------------------------------
Epoch 3
Loss in train set: 0.465264
Loss in validation set: 0.456809, Accuracy 84.750%
--------------------------------------------------
Epoch 4
Loss in train set: 0.459294
Loss in validation set: 0.454136, Accuracy 85.152%
--------------------------------------------------
Epoch 5
Loss in train set: 0.457008
Loss in validation set: 0.452364, Accuracy 85.252%
--------------------------------------------------
Epoch 6
Loss in train set: 0.454927
Loss in validation set: 0.452040, Accuracy 85.272%
--------------------------------------------------
Epoch 7
Loss in train set: 0.453844
Loss in validation set: 0.450862, Accuracy 85.333%
--------------------------------------------------
Epoch 8
Loss in train set: 0.45249

### Optuna in Tensorflow

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val = fe(X_train, X_val)

train_data = np.column_stack([X_train, y_train.values.reshape(-1, 1)])
val_data = np.column_stack([X_val, y_val.values.reshape(-1, 1)])

def load_train(train):
    train = tf.data.Dataset.from_tensor_slices(train)
    train = train.shuffle(10000)
    train = train.map(lambda x: (x[:-1], x[-1]))
    return train.batch(32).prefetch(1)

def load_val(val):
    val = tf.data.Dataset.from_tensor_slices(val)
    val = val.map(lambda x: (x[:-1], x[-1]))
    return val.batch(32).prefetch(1)


class MyModelTensorFlow(Model):
    def __init__(self):
        super().__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(32, activation='relu')
        self.dense3 = layers.Dense(1, activation='sigmoid')
        self.dropout = layers.Dropout(0.2)
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dropout(x)
        x = self.dense3(x)
        return x

train =  load_train(train_data)
val = load_val(val_data)

model = MyModelTensorFlow()

early_stop = EarlyStopping(monitor='val_acc', patience=7, mode='max', restore_best_weights=True)
model.compile('adam', loss='binary_crossentropy', metrics='acc')
history = model.fit(train, epochs=100, validation_data=(val), callbacks=[early_stop])
print('-'*80)
print(f"Best accuracy: {max(history.history['val_acc']):.3f}")

Epoch 1/100
623/623 [==============================] - 5s 6ms/step - loss: 0.3780 - acc: 0.8249 - val_loss: 0.3250 - val_acc: 0.8523
Epoch 2/100
623/623 [==============================] - 4s 6ms/step - loss: 0.3266 - acc: 0.8470 - val_loss: 0.3217 - val_acc: 0.8525
Epoch 3/100
623/623 [==============================] - 4s 6ms/step - loss: 0.3188 - acc: 0.8493 - val_loss: 0.3171 - val_acc: 0.8553
Epoch 4/100
623/623 [==============================] - 4s 6ms/step - loss: 0.3126 - acc: 0.8529 - val_loss: 0.3149 - val_acc: 0.8555
Epoch 5/100
623/623 [==============================] - 4s 6ms/step - loss: 0.3103 - acc: 0.8532 - val_loss: 0.3163 - val_acc: 0.8557
Epoch 6/100
623/623 [==============================] - 4s 6ms/step - loss: 0.3076 - acc: 0.8553 - val_loss: 0.3136 - val_acc: 0.8559
Epoch 7/100
623/623 [==============================] - 4s 6ms/step - loss: 0.3040 - acc: 0.8572 - val_loss: 0.3142 - val_acc: 0.8551
Epoch 8/100
623/623 [==============================] - 4s 6ms/step - 

In [ ]:
def define_model(trial):
    num_layers = trial.suggest_int("num_layers", 1, 5)
    weight_decay = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)
    model = tf.keras.Sequential()
    for i in range(num_layers):
        n_units = trial.suggest_int('n_units_l{}'.format(i), 4, 128)
        model.add(layers.Dense(n_units, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
        prob = trial.suggest_float('dropout_l{}'.format(i), 0.2, 0.4)
        model.add(layers.Dropout(prob))
    model.add(layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    return model

In [ ]:
def define_optimizer(trial):
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    optimizer = getattr(tf.keras.optimizers, optimizer_name)(learning_rate=lr)
    return optimizer

In [ ]:
def learn(model, optimizer, dataset, mode='eval'):
    accuracy = tf.keras.metrics.BinaryAccuracy()
    for batch, (data, target) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = model(data, training=(mode == "train"))
            binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()
            loss_value = tf.reduce_mean(binary_cross_entropy(y_true=target, y_pred=logits))
            if mode == "eval":
                accuracy(
                    tf.cast(target, tf.int32), logits
                )
            else:
                grads = tape.gradient(loss_value, model.variables)
                optimizer.apply_gradients(zip(grads, model.variables))
    if mode == 'eval':
        return accuracy.result()

In [ ]:
def objective(trial):
    model = define_model(trial)
    optimizer = define_optimizer(trial)
    for epoch in range(20):
        learn(model, optimizer, train, mode='train')
        acc = learn(model, optimizer, val, mode='eval')
        trial.report(acc, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return acc

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=600)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2022-02-18 12:25:27,957] A new study created in memory with name: no-name-59b29f76-00d2-4111-9a09-b6b412b9d479
[I 2022-02-18 12:29:56,198] Trial 0 finished with value: 0.8482041954994202 and parameters: {'num_layers': 1, 'weight_decay': 4.148372512784383e-09, 'n_units_l0': 96, 'dropout_l0': 0.39938897842497756, 'optimizer': 'Adam', 'learning_rate': 1.1341478924715399e-05}. Best is trial 0 with value: 0.8482041954994202.
[I 2022-02-18 12:36:25,299] Trial 1 finished with value: 0.8486048579216003 and parameters: {'num_layers': 3, 'weight_decay': 1.1075709272781065e-05, 'n_units_l0': 73, 'dropout_l0': 0.39576221100782805, 'n_units_l1': 81, 'dropout_l1': 0.21499000809671132, 'n_units_l2': 78, 'dropout_l2': 0.21058591882381128, 'optimizer': 'Adam', 'learning_rate': 2.9644946659079335e-05}. Best is trial 1 with value: 0.8486048579216003.


Study statistics: 
  Number of finished trials:  2
  Number of pruned trials:  0
  Number of complete trials:  2
Best trial:
  Value:  0.8486048579216003
  Params: 
    num_layers: 3
    weight_decay: 1.1075709272781065e-05
    n_units_l0: 73
    dropout_l0: 0.39576221100782805
    n_units_l1: 81
    dropout_l1: 0.21499000809671132
    n_units_l2: 78
    dropout_l2: 0.21058591882381128
    optimizer: Adam
    learning_rate: 2.9644946659079335e-05


In [ ]:
model = define_model(trial)
early_stop = EarlyStopping(monitor='val_acc', patience=7, mode='max', restore_best_weights=True)
optimizer = getattr(tf.keras.optimizers, trial.params['optimizer'])(learning_rate=trial.params['learning_rate'])
model.compile(optimizer, loss='binary_crossentropy', metrics='acc')
history = model.fit(train, epochs=100, validation_data=(val), callbacks=[early_stop])
print('-'*127)
print(f"Best accuracy: {max(history.history['val_acc']):.3f}")

Epoch 1/100
623/623 [==============================] - 9s 11ms/step - loss: 0.5905 - acc: 0.7225 - val_loss: 0.5240 - val_acc: 0.7641
Epoch 2/100
623/623 [==============================] - 7s 11ms/step - loss: 0.5207 - acc: 0.7646 - val_loss: 0.4758 - val_acc: 0.7685
Epoch 3/100
623/623 [==============================] - 7s 12ms/step - loss: 0.4786 - acc: 0.7775 - val_loss: 0.4353 - val_acc: 0.7882
Epoch 4/100
623/623 [==============================] - 5s 8ms/step - loss: 0.4469 - acc: 0.7894 - val_loss: 0.4041 - val_acc: 0.8166
Epoch 5/100
623/623 [==============================] - 4s 7ms/step - loss: 0.4263 - acc: 0.8001 - val_loss: 0.3844 - val_acc: 0.8266
Epoch 6/100
623/623 [==============================] - 4s 7ms/step - loss: 0.4110 - acc: 0.8081 - val_loss: 0.3716 - val_acc: 0.8320
Epoch 7/100
623/623 [==============================] - 4s 7ms/step - loss: 0.4008 - acc: 0.8154 - val_loss: 0.3636 - val_acc: 0.8352
Epoch 8/100
623/623 [==============================] - 4s 7ms/step